# 🚇 서울 이벤트 발생과 혼잡도 연관성 분석
## 대규모 이벤트가 지하철 승하차 패턴 및 혼잡도에 미치는 영향

스포츠, 콘서트, 시위 등 도심 내 대규모 이벤트 발생 시 지하철 2·4·5호선의 데이터를 분석하여 혼잡도 변화를 정량적으로 검증합니다.

---

## 1. 분석 환경 및 라이브러리 로드
분석에는 Python의 데이터 분석 및 시각화 라이브러리를 활용합니다.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 한글 폰트 설정 (윈도우: Malgun Gothic / 맥: AppleGothic)
plt.rc('font', family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

## 2. 데이터 전처리 및 옵션 통합 (Preprocessing)
역명 정규화와 동일 날짜/역에서 발생하는 다중 이벤트를 처리합니다. 
분석의 정확도를 위해 역명에서 '역'이라는 글자를 제거하고, 괄호 안의 상세 명칭을 정리합니다.

In [ ]:
# 역명 정규화 예시 (역 접미사 및 괄호 제거)
def normalize_station(name):
    return name.replace('역', '').split('(')[0]

# 다중 이벤트 통합 (Option A: 카테고리 병합)
# [날짜, 호선, 역명]이 같은 경우 카테고리를 "야구, 행사" 형태로 통합
df_event = df_event.groupby(['날짜', '호선', '역명']).agg({
    '카테고리': lambda x: ', '.join(x.unique()),
    '인원': 'first'
}).reset_index()

3. 혼잡도 상승률(Congestion Rate) 계산

이벤트 미발생일의 동일 요일 평균 인원을 Baseline으로 하여 상승률을 정의합니다.

$$상승률(\%) = \frac{\text{당일 승하차합계} - \text{Baseline}}{\text{Baseline}} \times 100$$

In [ ]:
# Baseline 계산: 이벤트 제외 평일/주말 요일별 평균 승하차량
baseline = df_normal.groupby(['호선', '역명', '요일'])['승하차합계'].mean()

# 이벤트 당일과 Baseline 결합 후 상승률 산출
df_analysis['상승률'] = ((df_analysis['당일합계'] - df_analysis['Baseline']) / df_analysis['Baseline']) * 100

4. 주요 분석 결과 (Key Findings)
4.1 가설 검증: 이벤트는 혼잡도를 유발하는가?
분석 결과, 이벤트 발생 시 **85.3%**의 사례에서 혼잡도가 증가하여 가설이 입증되었습니다.

In [ ]:
is_increase = df_analysis['상승률'] > 0
print(f"혼잡도 증가 사례: {is_increase.mean()*100:.1f}%")

4.2 호선별 민감도 분석

5호선 (68.4%): 올림픽공원 대형 공연의 영향으로 민감도 1위.

2호선 (56.8%): 잠실 경기장 및 대규모 행사의 영향.

4호선 (0.1%): 고부하 노선 특성상 상대적 상승폭은 낮음.

4.3 상승률 TOP 5 고혼잡 사례

가장 높은 상승률을 보인 사례들은 모두 대형 행사(콘서트) 관련 사례였습니다.

순위,날짜,역명,카테고리,상승률
1,2023-06-17,2호선 종합운동장,행사,312.5%
2,2023-06-18,2호선 종합운동장,행사,286.4%
3,2024-06-22,5호선 올림픽공원,행사,275.2%

5. 결론 및 정책 제언

차별적 대응: 상승률이 200%를 상회하는 **'비일상적 초거대 혼잡'**에 자원 집중.

교통 협력: 대형 행사 시 주최측과 연계한 셔틀버스 운영 및 임시 열차 증편 필수.

상시 관리: 4호선과 같은 고부하 노선은 이벤트 여부와 무관한 상시 안전 요원 배치 중요.